### Link to homework
https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/01-docker-terraform/homework.md


In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from time import time

In [2]:
trip_data_file_path = "/workspaces/data-engineering-zoomcamp/data/green_tripdata_2019-10.csv.gz"
green_table_name = "green_tripdata_2019_10"

In [3]:
encoded_password = quote_plus("P@ssw0rd!")
print(encoded_password)
engine = create_engine(f"postgresql://postgres:{encoded_password}@db:5432/ny_taxi")

P%40ssw0rd%21


In [4]:
engine.connect()

In [5]:
query_up_to_1 = """
    SELECT *
    FROM green_tripdata_2019_10
    LIMIT 1
"""

pd.read_sql(query_up_to_1, engine)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1,2019-10-03 20:35:50,2019-10-03 21:07:53,N,1.0,62,91,2.0,5.1,24.0,0.5,0.5,0.0,0.0,None,0.3,25.3,1.0,1.0,0.0


## Question 1. Understanding docker first run 

Run docker with the `python:3.12.8` image in an interactive mode, use the entrypoint `bash`.

What's the version of `pip` in the image?

- 24.3.1
- 24.2.1
- 23.3.1
- 23.2.1

Answer: Answer: root@3b27cc74845d:/# pip --version
pip 24.3.1

## Question 2. Understanding Docker networking and docker-compose

Given the following `docker-compose.yaml`, what is the `hostname` and `port` that **pgadmin** should use to connect to the postgres database?

```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

- postgres:5433
- localhost:5432
- db:5433
- postgres:5432
- db:5432


##  Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from October 2019:

```bash
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz
```

You will also need the dataset with zones:

```bash
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
```

Download this data and put it into Postgres.

You can use the code from the course. It's up to you whether
you want to use Jupyter or a python script.

## Question 3. Trip Segmentation Count

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, **respectively**, happened:
1. Up to 1 mile
2. In between 1 (exclusive) and 3 miles (inclusive),
3. In between 3 (exclusive) and 7 miles (inclusive),
4. In between 7 (exclusive) and 10 miles (inclusive),
5. Over 10 miles 

Answers:

- 104,802;  197,670;  110,612;  27,831;  35,281
- 104,802;  198,924;  109,603;  27,678;  35,189
- 104,793;  201,407;  110,612;  27,831;  35,281
- 104,793;  202,661;  109,603;  27,678;  35,189
- 104,838;  199,013;  109,645;  27,688;  35,202



In [ ]:
query_up_to_1 = f"""
SELECT
  COUNT(GTD_UP_TO_1.trip_distance) AS UP_TO_1
FROM green_tripdata_2019_10 AS GTD_UP_TO_1
WHERE
  GTD_UP_TO_1."lpep_pickup_datetime" >= '2019-10-01 00:00:00'
  AND GTD_UP_TO_1."lpep_pickup_datetime" < '2019-11-01 00:00:00'
  AND GTD_UP_TO_1.trip_distance <= 1
"""

query_between_1_3 = f"""
SELECT COUNT(GTD.trip_distance) AS GTD
FROM green_tripdata_2019_10 AS GTD
WHERE
  GTD."lpep_pickup_datetime" >= '2019-10-01 00:00:00'
  AND GTD."lpep_pickup_datetime" < '2019-11-01 00:00:00'
  AND GTD.trip_distance > 1 AND GTD.trip_distance <= 3
"""

# query = f"""
# SELECT COUNT(GTD_UP_TO_1.trip_distance) AS UP_TO_1
# FROM green_tripdata_2019_10 AS GTD_UP_TO_1
# WHERE
#   GTD_UP_TO_1.trip_distance <= 1
# """

print(f"Up to 1: {pd.read_sql_query(query_up_to_1, engine)}")
print(f"Between 1 and 3: {pd.read_sql_query(query_between_1_3, engine)}")

## Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance?
Use the pick up time for your calculations.

Tip: For every day, we only care about one single trip with the longest distance. 

- 2019-10-11
- 2019-10-24
- 2019-10-26
- 2019-10-31

In [ ]:
query_up_to_1 = f"""
SELECT COUNT(*) FROM {green_table_name}
"""

pd.read_sql_query(query_up_to_1, engine)

,count
0,476386


In [ ]:
query_up_to_1 = f"""
SELECT * FROM green_tripdata_2019_10 AS GTD
ORDER BY GTD.trip_distance DESC
LIMIT 1
"""

pd.read_sql_query(query_up_to_1, engine)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-10-31 23:23:41,2019-11-01 13:01:07,N,5.0,129,265,1.0,515.89,100.0,2.75,0.0,0.0,0.0,None,0.3,103.05,2.0,1.0,0.0


Answer: the longest trip was on 2019-10-31

### Question 5. Three biggest pick up Boroughs (WRONG SOLUTION)
Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

"Brooklyn" "Manhattan" "Queens";
"Bronx" "Brooklyn" "Manhattan";
"Bronx" "Manhattan" "Queens";
"Brooklyn" "Queens" "Staten Island";

In [ ]:
query_up_to_1 = f"""
SELECT SUM(total_amount) AS total_amount, 'Brooklyn/Manhattan/Queens' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Brooklyn' OR T.Borough = 'Manhattan' OR T.Borough = 'Queens')

UNION

SELECT SUM(total_amount) AS total_amount, 'Bronx/Brooklyn/Manhattan' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Bronx' OR T.Borough = 'Brooklyn' OR T.Borough = 'Manhattan')

UNION

SELECT SUM(total_amount) AS total_amount, 'Bronx/Manhattan/Queens' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Bronx' OR T.Borough = 'Manhattan' OR T.Borough = 'Queens')

UNION

SELECT SUM(total_amount) AS total_amount, 'Brooklyn/Queens/Staten Island' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Brooklyn' OR T.Borough = 'Queens' OR T.Borough = 'Staten Island')
"""
# --

pd.read_sql(query_up_to_1, engine)

,total_amount,group_name
0,221434.63,Bronx/Brooklyn/Manhattan
1,268136.34,Brooklyn/Manhattan/Queens
2,176207.63,Brooklyn/Queens/Staten Island
3,204633.19,Bronx/Manhattan/Queens


### Question 5. Three biggest pick up Boroughs (CORRECT SOLUTION)
Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

"Brooklyn" "Manhattan" "Queens";
"Bronx" "Brooklyn" "Manhattan";
"Bronx" "Manhattan" "Queens";
"Brooklyn" "Queens" "Staten Island";

In [ ]:
query_up_to_1 = f"""
SELECT TZ."borough", SUM(total_amount) AS total_amount
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS TZ ON GTD."PULocationID" = TZ."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (TZ.borough = 'Brooklyn'
    OR TZ.borough = 'Manhattan'
    OR TZ.borough = 'Queens'
    OR TZ.borough = 'Bronx'
    OR TZ.borough = 'Staten Island')
GROUP BY TZ."borough"
ORDER BY SUM(total_amount) DESC
"""
# --

pd.read_sql(query_up_to_1, engine)

,borough,total_amount
0,Brooklyn,96333.24
1,Manhattan,92271.30
2,Queens,79531.80
3,Bronx,32830.09
4,Staten Island,342.59


Answer to question 5 is "Brooklyn" "Manhattan" "Queens"

### Question 6. Largest tip
For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

Note: it's not a typo, it's tip , not trip

Central Park
Jamaica
JFK Airport
Long Island City/Queens Plaza

In [ ]:
query_up_to_1 = f"""
SELECT *
FROM taxi_zones AS TZ
WHERE TZ."zone" = 'Astoria'
LIMIT 10
"""

pd.read_sql(query_up_to_1, engine)

,zone,LocationID,borough
0,Astoria,7,Queens


In [ ]:
query_up_to_1 = f"""
SELECT GTD."DOLocationID"
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS TZ ON GTD."PULocationID" = TZ."LocationID"
WHERE TZ."zone" = 'Astoria'
ORDER BY GTD."tip_amount" DESC
LIMIT 1
"""

pd.read_sql(query_up_to_1, engine)

,DOLocationID
0,132


In [ ]:
query_up_to_1 = f"""
WITH top_tip_location_id (location_id) AS (
  SELECT GTD."DOLocationID"
  FROM green_tripdata_2019_09 AS GTD
  INNER JOIN taxi_zones AS TZ ON GTD."PULocationID" = TZ."LocationID"
  WHERE TZ."zone" = 'Astoria'
  ORDER BY GTD."tip_amount" DESC
  LIMIT 1
)
SELECT TZ.zone
FROM taxi_zones AS TZ, top_tip_location_id AS TTLI
WHERE TZ."LocationID" = TTLI."location_id"
"""

pd.read_sql(query_up_to_1, engine)

,zone
0,JFK Airport
